# Vision and Cognitive Systems - Project

<a href="https://colab.research.google.com/github/GianmarcoLattaruolo/Vision_Project/blob/main/Vision_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First of all let'smount our vision drive.

In [ ]:
# with this line we can check if we are in colab or not
import sys
in_colab = 'google.colab' in sys.modules
print("are we in Colab?:",in_colab)
if in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    os.chdir(r'/content/drive/MyDrive/GeoEstimation')
    print(os.getcwd())
    !python --version
    !pip install pandas==1.1.5
    !pip install msgpack==1.0.0
    !pip install tqdm==4.50
    !pip install torch==1.6
    !pip install torchvision==0.7
    !pip install torchtext==0.8.0 torch==1.6 pytorch-lightning==1.0.1 
    !pip install s2sphere==0.2.5

    #we lack only this to packages
    #!pip install yaml==0.2.5
    #!pip install cudatoolkit==10.2
else:
    import os
    current_wd = os.getcwd()
    if current_wd.split('\\')[-1] == 'Vision_Project':
        os.chdir(r'GeoEstimation')
    sys.path.append(current_wd + r'\GeoEstimation')

Now we need to install some specific packages with certain version to account for the original environment in which the paper results were obtained. We need to match these conditions:
```python
  - python=3.8
  - msgpack-python=1.0.0
  - pandas=1.1.5
  - yaml=0.2.5
  - tqdm=4.50
  - cudatoolkit=10.2
  - pytorch=1.6
  - torchvision=0.7
  - pytorch-lightning=1.0.1
  - pip
  - pip:
    - s2sphere==0.2.5
```

In [18]:
if in_colab:
    import msgpack
    #print("msgpack:", msgpack.__version__) #msgpack sembra non avere l'attributo __version__ nella versiona in cui lo abbiamo installato
    import pandas
    print("pandas:",pandas.__version__)
    import yaml
    print("yaml:",yaml.__version__)
    import tqdm
    print("tqdm:",tqdm.__version__)
    #import cudatoolkit
    #print("cudatoolkit:",cudatoolkit.__version__)
    import torch
    print("pytorch:",torch.__version__)
    import torchvision
    print("torchvision:",torchvision.__version__)
    import pytorch_lightning as pl
    print("pytorch_lightning:",pl.__version__)

pandas: 1.1.5
yaml: 6.0
tqdm: 4.64.1
pytorch: 1.13.1+cu116
torchvision: 0.7.0
pytorch_lightning: 1.0.1


# Reproduce paper results


In [ ]:
from pathlib import Path
from math import ceil

import pandas as pd
import torch
import pytorch_lightning as pl

from classification.train_base import MultiPartitioningClassifier # class defining our model
from classification.dataset import FiveCropImageDataset # class for preparing the images before giving them to the NN

## Load the model

In [ ]:
# where model's params and hyperparams are saved
checkpoint = "models/base_M/epoch=014-val_loss=18.4833.ckpt"
hparams = "models/base_M/hparams.yaml"

In [ ]:
# load_from_checkpoint is a static method from pytorch lightning, inherited by MultiPartitioningClassifier
# it permits to load a model previously saved, in the form of a checkpoint file, and one with hyperparameters
# MultiPartitioningClassifier is the class defining our model
model = MultiPartitioningClassifier.load_from_checkpoint(
    checkpoint_path=checkpoint,
    hparams_file=hparams,
    map_location=None
)

#model.eval() to see the model's structure

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\latta/.cache\torch\hub\checkpoints\resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:29<00:00, 3.48MB/s]


In [ ]:
#to allow GPU
want_gpu = True
if want_gpu and torch.cuda.is_available():
    gpu = 1
else:
    gpu = None

# the class Trainer from pythorch lightining is the one responsible for training a deep NN
# it can initialize the model, run forward and backward passes, optimize, print stats, early stop...
wanted_precision = 32 #16 for half precision (how many bits for each number)
trainer = pl.Trainer(gpus=gpu, precision=wanted_precision)

GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores


## Load and initialize the images

In [ ]:
# where images are saved
image_dir = "resources/images/im2gps"
meta_csv = "resources/images/im2gps_places365.csv"

In [ ]:
#FiveCropImageDataset is the class for preparing the images before giving them to the NN
# in particular, it creates five different crops for every image
dataset = FiveCropImageDataset(meta_csv, image_dir)

Read resources/images/im2gps_places365.csv


In [ ]:
batch_size = 64
dataloader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=ceil(batch_size / 5),  #you divide by 5 because for each image you generate 5 different crops
                    shuffle=False,
                    num_workers=4 #number ot threads used for parallelism (cores of CPU?)
                )

## Run the model on the test set

In [ ]:
results = trainer.test(model, test_dataloaders=dataloader, verbose=False)

Testing: 100%|██████████| 19/19 [05:05<00:00, 16.07s/it]


c:\ProgramData\Anaconda3\envs\vs-project\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: The testing_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)


## Look at the results

In [ ]:
# formatting results into a pandas dataframe
df = pd.DataFrame(results[0]).T
#df["dataset"] = image_dir
df["partitioning"] = df.index
df["partitioning"] = df["partitioning"].apply(lambda x: x.split("/")[-1])
df.set_index(keys=["partitioning"], inplace=True) #keys=["dataset", "partitioning"] in case
print(df)

                  1         25        200       750       2500
partitioning                                                  
coarse        0.092827  0.312236  0.493671  0.666667  0.789030
middle        0.139241  0.345992  0.481013  0.679325  0.789030
fine          0.156118  0.388186  0.489451  0.662447  0.784810
hierarchy     0.147679  0.379747  0.497890  0.687764  0.793249


In [ ]:
# to save the dataframe on a csv file
fout = 'test_results.csv'
df.to_csv(fout)

This two commands give the same error:
```python
OSError: /usr/local/lib/python3.8/dist-packages/torchtext/_torchtext.so: undefined symbol: _ZN3c104impl23ExcludeDispatchKeyGuardC1ENS_14DispatchKeySetE
```

In [43]:
#Inference with pre-trained model:
!python3 -m classification.inference --image_dir resources/images/im2gps/
print("\n\n\n")
#Test on Already Trained Model
!python -m classification.test

Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/content/drive/MyDrive/GeoEstimation/classification/inference.py", line 8, in <module>
    from classification.train_base import MultiPartitioningClassifier
  File "/content/drive/MyDrive/GeoEstimation/classification/train_base.py", line 10, in <module>
    import pytorch_lightning as pl
  File "/usr/local/lib/python3.8/dist-packages/pytorch_lightning/__init__.py", line 56, in <module>
    from pytorch_lightning.core import LightningDataModule, LightningModule
  File "/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/__init__.py", line 14, in <module>
    from pytorch_lightning.core.datamodule import LightningDataModule
  File "/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/datamodule.py", line 22, in <module>
    

These are the other commands for Training from Scratch (that we won't use maybe):

```python
# download and preprocess images
wget https://github.com/TIBHannover/GeoEstimation/releases/download/v1.0/mp16_urls.csv -O resources/mp16_urls.csv
wget https://github.com/TIBHannover/GeoEstimation/releases/download/pytorch/yfcc25600_urls.csv -O resources/yfcc25600_urls.csv 
python download_images.py --output resources/images/mp16 --url_csv resources/mp16_urls.csv --shuffle
python download_images.py --output resources/images/yfcc25600 --url_csv resources/yfcc25600_urls.csv --shuffle --size_suffix ""

# assign cell(s) for each image using the original meta information
wget https://github.com/TIBHannover/GeoEstimation/releases/download/v1.0/mp16_places365.csv -O resources/mp16_places365.csv
wget https://github.com/TIBHannover/GeoEstimation/releases/download/pytorch/yfcc25600_places365.csv -O resources/yfcc25600_places365.csv
python partitioning/assign_classes.py
# remove images that were not downloaded 
python filter_by_downloaded_images.py

# train geo model from scratch
python -m classification.train_base --config config/baseM.yml
```

In [25]:
os.chdir(r'/content/drive/MyDrive/GeoEstimation/resources/images/im2gps')
print(len(os.listdir()))
os.chdir(r'/content/drive/MyDrive/GeoEstimation')
print(os.getcwd())
import torch
print(torch.cuda.is_available())

# Output would be True if Pytorch is using GPU otherwise it would be False.
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

237
/content/drive/MyDrive/GeoEstimation
True
1
Tesla T4


In [ ]:
#@title
#libraries to import
#known
import pandas as pd
import numpy as np
import os
import re
import torchvision
import torch
import PIL
from PIL import Image
from PIL import ImageFile
import sys
import time
from math import ceil



#Unknown
from typing import Union
from io import BytesIO
import random
from argparse import Namespace, ArgumentParser
from pathlib import Path
from multiprocessing import Pool
from functools import partial
import requests
import logging
import json
import yaml
from tqdm.auto import tqdm
#from classification.train_base import MultiPartitioningClassifier
#from classification.dataset import FiveCropImageDataset

#to divide
from classification import utils_global
from classification.s2_utils import Partitioning, Hierarchy
from classification.dataset import MsgPackIterableDatasetMultiTargetWithDynLabels


The main link and paper that we need to follow is [this](https://github.com/TIBHannover/GeoEstimation) and [this](https://github.com/TIBHannover/GeoEstimation/releases/) for the pretrained models.

Davide ha trovato questo che forse è meglio [kaggle](https://www.kaggle.com/code/habedi/inspect-the-dataset/data)

[Qui](https://qualinet.github.io/databases/image/world_wide_scale_geotagged_image_dataset_for_automatic_image_annotation_and_reverse_geotagging/) ci sono dei links che potrebbero essere usati con colab col comando !wget.